22/06/2023      
notebook     
objectif : évaluer les performances de EL sur des textes contemporains    
contexte : suite à la réunion du 02/06/2023

# Evaluation comparative  du système d'entity linking de SpaCy

In [1]:
import spacy
import glob
import pandas as pd
import os
import time
import json
from art import *

def lire_fichier(chemin):
    with open(chemin, "r", encoding="utf-8") as fichier:
        texte = fichier.read()
        return texte

def stocker(chemin, contenu):
    with open(chemin, "w") as fichier:
        json.dump(contenu, fichier, indent=2)

def stocker_tex(name, contenu):
    with open(name, "w") as file:
        file.write(contenu)

def tableau(data):
    df = pd.DataFrame(data)
    table_latex = df.to_latex(index=False)
    print(table_latex)
    #stocker_tex("txts-contemporains.tex", df.to_latex(index=False))
    #stocker("txts-contemporains.json", df.to_dict('index'))

t0 = time.time()  
bilan = []
def process (texte):
    i = 1
    models = ["fr_core_news_sm","fr_core_news_lg","en_core_web_sm", "en_core_web_lg", "en_core_web_md"]
    for model in models :
        tprint (model, font="cybermedium")
        nlp = spacy.load(model)
        # add pipeline (declared through entry_points in setup.py)
        nlp.add_pipe("entityfishing")
        doc = nlp(texte)
        # returns all entities in the whole document
        # iterates over sentences and prints linked entities
        for ent in doc.ents:
                ide = "ID"+str(i)
                dico_sent_tok[ide]={}
                dico_sent_tok[ide]["label"]=ent.label_
                #dico_sent_tok[ide]["description"]=description
                dico_sent_tok[ide]["span"]=ent.text
                dico_sent_tok[ide]["score"]=ent._.nerd_score
                dico_sent_tok[ide]["URL"]=ent._.url_wikidata
                dico_sent_tok[ide]["ID"]=ent._.kb_qid
                i+=1
    print (dico_sent_tok) 

2023-08-16 00:53:06.811242: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-16 00:53:07.537980: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def read (texte):
    models = ["fr_core_news_sm","fr_core_news_lg","en_core_web_sm", "en_core_web_lg", "en_core_web_md"]
    for model in models :
        tprint (model, font="cybermedium")
        nlp = spacy.load("en_core_web_md")
        # add pipeline (declared through entry_points in setup.py)
        nlp.add_pipe("entityfishing")
        doc = nlp(texte)
        # returns all entities in the whole document
        # iterates over sentences and prints linked entities
        for ent in doc.ents:
            print((ent.text, ent.label_, ent._.kb_qid, ent._.url_wikidata, ent._.nerd_score))

In [3]:
def latexiser_fr(texte):
    tab_entite =[]
    nlp = spacy.load("fr_core_news_md")
    
    nlp.add_pipe("entityfishing")
    doc = nlp(texte)
        # returns all entities in the whole document
        # iterates over sentences and prints linked entities
    for ent in doc.ents:
        tab_entite.append({
            #"modèle": mod,
            "span" : ent.text,
            "label " : ent.label_,
            "URL" : ent._.url_wikidata,
            #"ID" : ent._.kb_qid,
            "score" : ent._.nerd_score,
            })          
    df = pd.DataFrame(tab_entite)
    table_latex = df.to_latex(index=False)
    latex = df.to_latex(index=False)
    print(latex)

## Avec un extrait de la page Wikipédia "Discord"

### En français

Discord est un logiciel propriétaire gratuit de VoIP et de messagerie instantanée. Il fonctionne sur les systèmes d’exploitation Windows, macOS, Linux, Android, iOS ainsi que sur les navigateurs web. "

In [4]:
extrait = "Discord est un logiciel propriétaire gratuit de VoIP et de messagerie instantanée. Il fonctionne sur les systèmes d’exploitation Windows, macOS, Linux, Android, iOS ainsi que sur les navigateurs web.  "

In [5]:
latexiser_fr(extrait)

\begin{tabular}{lllr}
\toprule
   span & label  &                                    URL &  score \\
\midrule
Discord &   MISC & https://www.wikidata.org/wiki/Q5281690 & 0.6800 \\
   VoIP &   MISC &   https://www.wikidata.org/wiki/Q81945 & 0.9026 \\
Windows &   MISC &    https://www.wikidata.org/wiki/Q1406 & 0.6574 \\
  macOS &   MISC &   https://www.wikidata.org/wiki/Q14116 & 0.9161 \\
  Linux &   MISC &   https://www.wikidata.org/wiki/Q14579 & 0.8570 \\
Android &   MISC &      https://www.wikidata.org/wiki/Q94 & 0.7389 \\
    iOS &   MISC &   https://www.wikidata.org/wiki/Q48493 & 0.9124 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_9106/1147971263.py:19: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  table_latex = df.to_latex(index=False)
/tmp/ipykernel_9106/1147971263.py:20: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = df.to_latex(index=False)


In [20]:
read(extrait)

____ ____     ____ ____ ____ ____     _  _ ____ _ _ _ ____     ____ _  _ 
|___ |__/     |    |  | |__/ |___     |\ | |___ | | | [__      [__  |\/| 
|    |  \ ___ |___ |__| |  \ |___ ___ | \| |___ |_|_| ___] ___ ___] |  | 
                                                                         

('Discord est un', 'ORG', None, None, None)
('Il fonctionne sur les systèmes d’exploitation Windows', 'ORG', None, None, None)
('macOS', 'ORG', 'Q14116', 'https://www.wikidata.org/wiki/Q14116', 0.9161)
('Linux', 'ORG', 'Q14579', 'https://www.wikidata.org/wiki/Q14579', 0.857)
('Android', 'ORG', 'Q94', 'https://www.wikidata.org/wiki/Q94', 0.7389)
____ ____     ____ ____ ____ ____     _  _ ____ _ _ _ ____     _    ____ 
|___ |__/     |    |  | |__/ |___     |\ | |___ | | | [__      |    | __ 
|    |  \ ___ |___ |__| |  \ |___ ___ | \| |___ |_|_| ___] ___ |___ |__] 
                                                                         

('Discord est un', 'ORG', None, None, None)
('Il fonctionne

In [ ]:
\footnotesize{
\begin{tabular}{|p{2cm}|p{1cm}|p{1cm}|p{3cm}|p{2cm}|p{1cm}|}
\hline
span
& label SpaCy
& label wikidata
& défintion wikidata
& url
& certitude
\\ \hline
Discord est un
& 'ORG'
& NA
& NA
& NA
& NA
\\ \hline
Il fonctionne sur les systèmes d’exploitation Windows
& ORG
& NA
& NA
& NA
& NA
\\ \hline
macOS
& ORG
& macOS
& operating system for Apple computers
& \url{https://www.wikidata.org/wiki/Q14116}
& 0.9161
\\ \hline
Linux
& ORG
& Linux kernel
& Unix-like operating system kernel, basis for all Linux operating systems / Linux distributions
& \url{https://www.wikidata.org/wiki/Q14579}
& 0.857
\\ \hline
Android
& ORG
& Android
& open-source operating system for mobile devices created by Google
& \url{https://www.wikidata.org/wiki/Q94}
& 0.7389
\\ \hline
\end{tabular}} 

## En anglais

Discord is an American VoIP and instant messaging social platform. Discord runs on Windows, macOS, Android, iOS, iPadOS, Linux, and in web browsers. 

In [21]:
discord = "Discord is an American VoIP and instant messaging social platform. Discord runs on Windows, macOS, Android, iOS, iPadOS, Linux, and in web browsers."

In [22]:
read(discord)

____ ____     ____ ____ ____ ____     _  _ ____ _ _ _ ____     ____ _  _ 
|___ |__/     |    |  | |__/ |___     |\ | |___ | | | [__      [__  |\/| 
|    |  \ ___ |___ |__| |  \ |___ ___ | \| |___ |_|_| ___] ___ ___] |  | 
                                                                         

('Discord', 'ORG', 'Q5281690', 'https://www.wikidata.org/wiki/Q5281690', 0.68)
('American', 'NORP', 'Q30', 'https://www.wikidata.org/wiki/Q30', 0.429)
('Discord', 'ORG', 'Q5281690', 'https://www.wikidata.org/wiki/Q5281690', 0.68)
('Windows', 'PRODUCT', 'Q1406', 'https://www.wikidata.org/wiki/Q1406', 0.6574)
('macOS', 'ORG', 'Q14116', 'https://www.wikidata.org/wiki/Q14116', 0.9161)
('Android', 'ORG', 'Q94', 'https://www.wikidata.org/wiki/Q94', 0.7389)
('iOS', 'ORG', 'Q48493', 'https://www.wikidata.org/wiki/Q48493', 0.9124)
('iPadOS', 'ORG', 'Q64350339', 'https://www.wikidata.org/wiki/Q64350339', 0.9148)
('Linux', 'ORG', 'Q14579', 'https://www.wikidata.org/wiki/Q14579', 0.857)
____ ____     ____ 

In [ ]:
\footnotesize{
\begin{tabular}{|p{2cm}|p{1cm}|p{1cm}|p{3cm}|p{2cm}|p{1cm}|}
\hline
span
& label SpaCy
& label wikidata
& défintion wikidata
& url
& certitude
\\ \hline
Discord
& ORG
& Discord
& 1933 film by Henry Edwards
& \url {https://www.wikidata.org/wiki/Q5281690}
& 0.68
\\ \hline
American
& NORP
& United States of America
& country in North America
& \url{https://www.wikidata.org/wiki/Q30}
& 0.429
\\ \hline
Windows
& PRODUCT
&  Microsoft Windows
& family of computer operating systems developed by Microsoft
& https://www.wikidata.org/wiki/Q1406
& \url{0.6574}
\\ \hline
macOS
& ORG
&  macOS
& operating system for Apple computers
& \url{https://www.wikidata.org/wiki/Q14116}
& 0.9161
\\ \hline
Android
& ORG
& Android (Q94)
& open-source operating system for mobile devices created by Google
& \url{https://www.wikidata.org/wiki/Q94}
& 0.7389
\\ \hline
iOS
& ORG
& iOS 
& mobile operating system by Apple Inc.
& \url{https://www.wikidata.org/wiki/Q48493}
& 0.9124
\\ \hline
iPadOS
& ORG
& iPadOS 
& mobile operating system of the iPad, developed by Apple Inc.
& \url{https://www.wikidata.org/wiki/Q64350339}
& 0.9148
\\ \hline
Linux
& ORG
& Linux kernel
& Unix-like operating system kernel, basis for all Linux operating systems / Linux distributions
& \url{https://www.wikidata.org/wiki/Q14579}
& 0.857

\\ \hline
\end{tabular}} 

## extrait de la page Wikipédia "Nabilla"

En novembre 2013, elle est la vedette de sa propre émission de téléréalité, Allô Nabilla, ce qui est une première en France, le concept étant repris des États-Unis où l’on suit la vie des stars, notamment comme avec Kesha, Paris Hilton ou les Kardashian. 

In [6]:
nabilla ="En novembre 2013, elle est la vedette de sa propre émission de téléréalité, Allô Nabilla, ce qui est une première en France, le concept étant repris des États-Unis où l’on suit la vie des stars, notamment comme avec Kesha, Paris Hilton ou les Kardashian. "

In [7]:
latexiser_fr(nabilla)

\begin{tabular}{lllr}
\toprule
        span & label  &                                   URL &  score \\
\midrule
Allô Nabilla &    PER &                                  None &    NaN \\
      France &    LOC &    https://www.wikidata.org/wiki/Q142 & 0.6702 \\
  États-Unis &    LOC &     https://www.wikidata.org/wiki/Q30 & 0.4895 \\
       Kesha &    ORG &  https://www.wikidata.org/wiki/Q33605 & 0.7583 \\
Paris Hilton &    ORG &  https://www.wikidata.org/wiki/Q47899 & 0.7640 \\
  Kardashian &    LOC & https://www.wikidata.org/wiki/Q186304 & 0.3953 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_9106/1147971263.py:19: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  table_latex = df.to_latex(index=False)
/tmp/ipykernel_9106/1147971263.py:20: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex = df.to_latex(index=False)


In [8]:
read(nabilla)

____ ____     ____ ____ ____ ____     _  _ ____ _ _ _ ____     ____ _  _ 
|___ |__/     |    |  | |__/ |___     |\ | |___ | | | [__      [__  |\/| 
|    |  \ ___ |___ |__| |  \ |___ ___ | \| |___ |_|_| ___] ___ ___] |  | 
                                                                         

('novembre 2013', 'DATE', None, None, None)
('elle est la vedette de sa', 'LOC', None, None, None)
('Allô Nabilla', 'PERSON', None, None, None)
('France', 'GPE', 'Q142', 'https://www.wikidata.org/wiki/Q142', 0.4631)
('des États-Unis où l’on', 'PERSON', None, None, None)
('comme avec Kesha', 'PERSON', None, None, None)
('Paris', 'GPE', 'Q90', 'https://www.wikidata.org/wiki/Q90', 0.4627)
____ ____     ____ ____ ____ ____     _  _ ____ _ _ _ ____     _    ____ 
|___ |__/     |    |  | |__/ |___     |\ | |___ | | | [__      |    | __ 
|    |  \ ___ |___ |__| |  \ |___ ___ | \| |___ |_|_| ___] ___ |___ |__] 
                                                                         

('novembre 2013',

In [ ]:
\footnotesize{
\begin{tabular}{|p{2cm}|p{1cm}|p{1cm}|p{3cm}|p{2cm}|p{1cm}|}
\hline
span
& label SpaCy
& label wikidata
& défintion wikidata
& url
& certitude
\\ \hline
novembre 2013
& DATE
& None
& None
& None
& None
& None
\\ \hline
elle est la vedette de sa
& LOC
& None
& None
& None
& None
& None
\\ \hline
Allô Nabilla
& PERSON
& None
& None
& None
\\ \hline
France
& GPE
& France
& country in Western Europe
&\url{https://www.wikidata.org/wiki/Q142}
& 0.4631
\\ \hline
des États-Unis où l’on
& PERSON
& None
& None
& None
& None
& None
\\ \hline
comme avec Kesha
& PERSON
& None
& None
& None
& None
& None
\\ \hline
Paris
& GPE
& Paris 
& capital and most populous city of France
& \url{https://www.wikidata.org/wiki/Q90}
& 0.4627
\\ \hline

## Flaubert

In [4]:
Flaubert= "Il recevait du fond de l'Allemagne ou de l'Italie une toile achetée à Paris quinze cents francs"

In [5]:
latexiser_fr(Flaubert)

\begin{tabular}{lllr}
\toprule
span & label  & URL & score \\
\midrule
Allemagne & LOC & https://www.wikidata.org/wiki/Q473295 & 0.347500 \\
l'Italie & LOC & NaN & NaN \\
Paris & LOC & https://www.wikidata.org/wiki/Q90 & 0.462700 \\
\bottomrule
\end{tabular}

